In [3]:
from ortools.linear_solver import pywraplp
import pandas as pd
import math

### Links

https://kunlei.github.io/python-ortools-notes/

In [4]:
# Load data from the Excel file
file_path = "../data/VOSimu-InputInformation.xlsx"

locations_df = pd.read_excel(file_path, sheet_name="Locations")
vehicles_df = pd.read_excel(file_path, sheet_name="Vehicles")
orders_df = pd.read_excel(file_path, sheet_name="ContainerOrders")

In [5]:
orders_df

,TractorOrderId,ContainerOrderId,ContainerName,Length,OriginLocation,DestinationLocation,Time first known
0,TO_CO_TFTU000067,CO_TFTU000067,TFTU000067,20,QC005,WS001.01,2024-08-21 07:12:00
1,TO_CO_TFTU000121,CO_TFTU000121,TFTU000121,40,QC003,WS001.01,2024-08-21 07:21:00
2,TO_CO_TFTU000156,CO_TFTU000156,TFTU000156,40,QC006,WS001.01,2024-08-21 07:28:00
3,TO_CO_TFTU000175,CO_TFTU000175,TFTU000175,40,QC003,WS001.01,2024-08-21 07:35:00
4,TO_CO_TFTU000211,CO_TFTU000211,TFTU000211,40,QC005,WS001.01,2024-08-21 07:45:00
...,...,...,...,...,...,...,...
281,TO_CO_TFTU000151,CO_TFTU000151,TFTU000151,20,QC006,YARD001.57,2024-08-21 07:27:00
282,TO_CO_TFTU000166,CO_TFTU000166,TFTU000166,20,QC003,YARD001.57,2024-08-21 07:31:00
283,TO_CO_TFTU000059,CO_TFTU000059,TFTU000059,20,QC006,YARD001.58,2024-08-21 07:11:00
284,TO_CO_TFTU000005,CO_TFTU000005,TFTU000005,20,QC003,YARD001.59,2024-08-21 07:01:00


In [ ]:
# Helper function to calculate Manhattan distance
def manhattan_distance(x1, y1, x2, y2):
    return abs(x1 - x2) + abs(y1 - y2)

# Parameters
VEHICLE_SPEED = 5545  # mm/s
PICKUP_DROPOFF_TIME = 60  # seconds

# Build location and vehicle data structures
locations = {row["Location Name"].strip(): (row["X-Coordinate [mm]"], row["Y-Coordinate [mm]"], row["Capacity limitation (# SC)"]) for _, row in locations_df.iterrows()}
vehicles = {row["ID"]: {"start_time": row["LogOn"], "end_time": row["LogOff"], "initial_location": row["StartLocation"]} for _, row in vehicles_df.iterrows()}
orders = []
for _, row in orders_df.iterrows():
    orders.append({
        "id": row["ContainerOrderId"],
        "origin": row["OriginLocation"].strip(),
        "destination": row["DestinationLocation"].strip(),
        "length": row["Length"],
        "time_first_known": row["Time first known"]
    })

# Create the solver
solver = pywraplp.Solver.CreateSolver('SCIP')
if not solver:
    raise Exception("SCIP solver is not available.")

# Define decision variables
x = {}
arrival_time = {}
for v in vehicles:
    for o in orders:
        x[v, o["id"]] = solver.IntVar(0, 1, f'x[{v},{o["id"]}]')  # 1 if vehicle v is assigned to order o, otherwise 0
        arrival_time[v, o["id"]] = solver.NumVar(0, solver.infinity(), f'arrival_time[{v},{o["id"]}]')

# Objective: Minimize total travel time (distance / speed) + waiting time + pickup/drop-off time
total_time = solver.Sum(
    (
        manhattan_distance(
            *locations[o["origin"]][:2], *locations[o["destination"]][:2]
        ) / VEHICLE_SPEED + 2 * PICKUP_DROPOFF_TIME  # Travel time + pickup and drop-off time
    ) * x[v, o["id"]]
    for v in vehicles for o in orders
)
solver.Minimize(total_time)

# Constraints
# 1. Each order must be assigned to exactly one vehicle
for o in orders:
    solver.Add(solver.Sum(x[v, o["id"]] for v in vehicles) == 1)

# 2. Capacity constraints at each location
for loc in locations:
    if locations[loc][2]:  # Only apply if capacity is limited
        cap = locations[loc][2]
        for t in range(24 * 60 * 60):  # Time slots in seconds (assuming max 24h schedule)
            solver.Add(
                solver.Sum(
                    x[v, o["id"]] for v in vehicles for o in orders
                    if o["origin"] == loc or o["destination"] == loc and arrival_time[v, o["id"]] <= t
                ) <= cap
            )

# 3. Vehicle availability constraints
for v in vehicles:
    start = vehicles[v]["start_time"]
    end = vehicles[v]["end_time"]
    for o in orders:
        solver.Add(x[v, o["id"]] * (o["time_first_known"] >= start) * (o["time_first_known"] <= end))

# Solve the model
status = solver.Solve()

# Output results
if status == pywraplp.Solver.OPTIMAL:
    print("Optimal solution found!")
    for v in vehicles:
        assigned_orders = [o["id"] for o in orders if x[v, o["id"]].solution_value() == 1]
        for o_id in assigned_orders:
            travel_time = manhattan_distance(
                *locations[orders[o_id - 1]["origin"]][:2], *locations[orders[o_id - 1]["destination"]][:2]
            ) / VEHICLE_SPEED
            print(f"Vehicle {v} assigned to order {o_id}, travel time: {travel_time:.2f}s")
    print(f"Total time: {total_time.solution_value()} seconds")
else:
    print("No optimal solution found.")
